In [ ]:
import os
import pickle
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
import logging

# Configurar el logger
logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

# Cargar el dataset
df = pd.read_csv('../history/HISTORY.csv')
df['Fechas'] = pd.to_datetime(df['Fechas'], format='%m/%d/%Y')
df.rename(columns={'Fechas': 'ds', 'Llamadas': 'y'}, inplace=True)

# Crear y ajustar el modelo Prophet
model = Prophet()
model.fit(df)

# Crear un DataFrame para las fechas futuras (3 meses futuros)
future = model.make_future_dataframe(periods=3, freq='M')

# Crear un DataFrame para las fechas pasadas (3 meses pasados)
past = pd.DataFrame({'ds': pd.date_range(start=df['ds'].min() - pd.DateOffset(months=6), 
                                         end=df['ds'].min() - pd.DateOffset(months=1), 
                                         freq='M')})

# Combinar ambos DataFrames
combined = pd.concat([past, future], ignore_index=True)

# Realizar la predicción
forecast = model.predict(combined)

# Crear un diccionario con los detalles del modelo Prophet
model_dict = {
    'model': model,
    'algorithm_name': 'Prophet',
    'author': 'Enrique',
    'dataset_info': {
        'num_records': len(df),
        'columns': list(df.columns)
    },
    'prediction_info': {
        'forecast_horizon': 3,  # meses futuros
        'past_window': 6        # meses pasados
    },
    'metrics': 'No aplica en Prophet',  # Prophet no calcula métricas como accuracy directamente
    'notes': 'Modelo para pronóstico de llamadas'
}

# Crear la ruta del archivo
output_dir = '../models'
output_file = 'prophet_model.pkl'

# Guardar el modelo en un archivo pickle
try:
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, output_file), 'wb') as file:
        pickle.dump(model_dict, file)
        print("El modelo se guardó correctamente en:", os.path.join(output_dir, output_file))
except Exception as e:
    logger.error(f"Ha ocurrido un error al guardar el modelo: {e}")

# Cargar el modelo desde el archivo pickle (para verificar)
try:
    with open(os.path.join(output_dir, output_file), 'rb') as file:
        loaded_model = pickle.load(file)
        print("El modelo se cargó correctamente:", loaded_model)
except Exception as e:
    logger.error(f"Ha ocurrido un error al cargar el modelo: {e}")

# Graficar los resultados
fig1 = model.plot(forecast)
plt.title('Pronóstico de llamadas utilizando Prophet')
plt.xlabel('Fecha')
plt.ylabel('Número de llamadas')
plt.show()

# Mostrar el pronóstico final (últimos 9 registros)
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(9))


/workspaces/Forecasts/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


FileNotFoundError: [Errno 2] No such file or directory: '../history/HISTORY.csv'